### 1. 단순한 기본형 봇 만들기

In [7]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os
import warnings
warnings.filterwarnings("ignore")

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = openai.api_key

In [8]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chains import LLMChain, ConversationChain
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

message = [
    SystemMessage(
        content = "당신은 사용자가 최적화된 투자 의사결정을 내리도록 도움을 주는 유용한 도우미입니다."
    ),
    HumanMessage(
        content = "저는 애플(AAPL)을 1년동안 투자하고 있는데, 앞으로 계속 추가 매수를 하는게 좋을까요?"
    )
]

In [9]:
output = chat.invoke(message)
print(output.content)

애플(AAPL)은 기술 기업으로서 현재 시장에서 안정적이고 성장하는 기업으로 평가받고 있습니다. 하지만 투자의사결정은 개인의 투자목표, 투자기간, 위험 선호도 등을 고려해야 합니다. 추가 매수를 고려할 때 고려해야 할 몇 가지 사항이 있습니다.

1. 재무제표 분석: 애플의 재무건전성과 수익성을 분석해야 합니다. 매출액, 순이익, 자산가치 등을 확인하여 기업의 건강한 재무 상태를 확인할 수 있습니다.

2. 시장 분석: 애플이 속한 산업과 시장의 트렌드를 파악해야 합니다. 기술 산업이 미래에 어떻게 변화할지 고려해야 합니다.

3. 기업 분석: 애플의 제품 라인업, 혁신력, 경쟁력 등을 분석하여 기업이 성장할 여지가 있는지 확인해야 합니다.

4. 주가 분석: 현재 애플의 주가가 적정한 가치에 있는지, 과도하게 고평가되었는지 등을 분석하여 추가 매수의 타이밍을 결정할 수 있습니다.

따라서, 추가 매수를 고려할 때에는 위와 같은 요인들을 종합적으로 고려하여 결정하는 것이 중요합니다. 또한, 투자에 대한 충분한 연구와 정보 수집을 통해 최선의 결정을 내리는 것이 중요합니다. 만약 추가로 도움이 필요하다면 금융 전문가와 상담을 하는 것을 권장합니다.


### 2. 기억 추가하기

이 시나리오는 비교적 짧은 메세지로 대화형 봇을 만드는 것이므로 `ConversationBufferMemory`로 충분할 것이다. 더 쉽게 구성할 수 있게 LLM과 메모리 컴포넌트를 결합하기 위해 `ConversationChain`도 초기화해 보자. 

In [10]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = chat,
    verbose = True,
    memory = memory
)

In [11]:
conversation.invoke("안녕하세요!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요!
AI:

> Finished chain.


{'input': '안녕하세요!', 'history': '', 'response': ' 안녕하세요! 어떻게 도와 드릴까요?'}

In [12]:
conversation.invoke('2025년에 성장가능성이 높은 산업은 무엇인가요?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요!
AI:  안녕하세요! 어떻게 도와 드릴까요?
Human: 2025년에 성장가능성이 높은 산업은 무엇인가요?
AI:

> Finished chain.


{'input': '2025년에 성장가능성이 높은 산업은 무엇인가요?',
 'history': 'Human: 안녕하세요!\nAI:  안녕하세요! 어떻게 도와 드릴까요?',
 'response': '2025년에는 인공지능, 사물인터넷, 빅데이터, 사물인터넷, 사이버 보안 및 생명 과학 분야가 성장 가능성이 높을 것으로 예상됩니다. 특히 인공지능과 빅데이터 분야는 계속해서 발전하여 많은 기회를 제공할 것으로 보입니다.'}

체인에서 볼 수 있듯이 이전 상호 작용을 추적하고 있다.

In [13]:
conversation.invoke('성장가능성이 희박한 산업도 말해 주세요')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요!
AI:  안녕하세요! 어떻게 도와 드릴까요?
Human: 2025년에 성장가능성이 높은 산업은 무엇인가요?
AI: 2025년에는 인공지능, 사물인터넷, 빅데이터, 사물인터넷, 사이버 보안 및 생명 과학 분야가 성장 가능성이 높을 것으로 예상됩니다. 특히 인공지능과 빅데이터 분야는 계속해서 발전하여 많은 기회를 제공할 것으로 보입니다.
Human: 성장가능성이 희박한 산업도 말해 주세요
AI:

> Finished chain.


{'input': '성장가능성이 희박한 산업도 말해 주세요',
 'history': 'Human: 안녕하세요!\nAI:  안녕하세요! 어떻게 도와 드릴까요?\nHuman: 2025년에 성장가능성이 높은 산업은 무엇인가요?\nAI: 2025년에는 인공지능, 사물인터넷, 빅데이터, 사물인터넷, 사이버 보안 및 생명 과학 분야가 성장 가능성이 높을 것으로 예상됩니다. 특히 인공지능과 빅데이터 분야는 계속해서 발전하여 많은 기회를 제공할 것으로 보입니다.',
 'response': '성장가능성이 희박한 산업으로는 전통적인 제조업, 일부 광업 및 석유화학 산업 등이 있습니다. 이러한 산업들은 기술 혁신과 경쟁력 부족으로 인해 미래에는 성장 가능성이 낮을 수 있습니다.'}

봇은 우리의 요청이 이전 답변과 관련이 있다는 것을 이해할 수 있다. 또한 `memory.load_memory_variables()` method를 사용해 메세지 이력을 검색할 수도 있다.

In [16]:
memory.load_memory_variables('성장가능성이 희박한 산업도 말해 주세요')

{'history': 'Human: 안녕하세요!\nAI:  안녕하세요! 어떻게 도와 드릴까요?\nHuman: 2025년에 성장가능성이 높은 산업은 무엇인가요?\nAI: 2025년에는 인공지능, 사물인터넷, 빅데이터, 사물인터넷, 사이버 보안 및 생명 과학 분야가 성장 가능성이 높을 것으로 예상됩니다. 특히 인공지능과 빅데이터 분야는 계속해서 발전하여 많은 기회를 제공할 것으로 보입니다.\nHuman: 성장가능성이 희박한 산업도 말해 주세요\nAI: 성장가능성이 희박한 산업으로는 전통적인 제조업, 일부 광업 및 석유화학 산업 등이 있습니다. 이러한 산업들은 기술 혁신과 경쟁력 부족으로 인해 미래에는 성장 가능성이 낮을 수 있습니다.'}

모든 상호 작용에서 `conversation.invoke` method를 실행하는 대신 `while` 반복문을 사용해 대화형으로 코딩할 수 있다.

In [19]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory


prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant that help the user to invest optimized stock portfolio."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [21]:
while True :
    query = input('input conversation (q to quit): ')
    if query == 'q':
        print('Quit the chatbot')
        break
    output = conversation({'question':query})
    print('user: ', query)
    print('AI system: ', output['text'])

user:  퀀텀 컴퓨팅 테마가 끝난 뒤 상승할 미국 주식 테마는 뭘까?
AI system:  퀀텀 컴퓨팅 테마가 끝나면 다양한 미국 주식 테마 중에서 성장 가능성이 있는 분야를 고려해볼 수 있습니다. 몇 가지 주목할 만한 테마들을 소개해드리겠습니다:

1. 인공지능(AI) 및 자동화 기술: 인공지능 기술은 다양한 산업 분야에서 혁신을 이끌고 있으며, 자동화 기술도 더 많은 기업들이 도입하고 있습니다. AI 및 자동화 기술 관련 기업들은 향후 성장 가능성이 높을 것으로 예상됩니다.

2. 사물인터넷(IoT) 및 스마트 시티: IoT 기술은 물류, 건설, 제조업 등 다양한 산업 분야에서 활용되고 있으며, 스마트 시티 관련 기술도 더 많은 관심을 받고 있습니다. 이러한 분야의 기업들이 향후 성장할 수 있는 가능성이 있습니다.

3. 친환경 및 신재생 에너지: 친환경 및 신재생 에너지 기술은 환경 보호 및 기후 변화 대응을 위해 더 많은 관심을 받고 있습니다. 태양광, 풍력 에너지 등의 신재생 에너지 기업들이 향후 성장할 수 있는 가능성이 있습니다.

이 외에도 의료기술, 클라우드 컴퓨팅, 사이버보안 등의 분야도 향후 성장 가능성이 있는 주식 테마로 고려될 수 있습니다. 투자를 고려하실 때에는 해당 분야의 시장 동향과 기업 분석을 통해 신중하게 결정하시는 것이 좋습니다.
user:  항공우주 관련 테마는 앞으로 상승할 가능성이 높을까?
AI system:  항공우주 관련 테마는 향후 상승할 가능성이 있다고 볼 수 있습니다. 여러 이유로 인해 항공우주 산업이 성장하는 추세에 있습니다.

1. 항공 여행 수요 증가: 코로나19로 인해 항공 여행이 감소했지만, 백신 보급과 여행 제한의 완화로 항공 여행 수요가 증가할 것으로 예상됩니다.

2. 항공기 수요 증가: 항공사들이 새로운 항공기를 도입하거나 기존 항공기를 교체하는 수요가 증가할 것으로 예상됩니다.

3. 우주 산업 발전: 우주 탐사, 우주 여행, 위성 발사 등의 우주 산업이 성장하고 있으며, 민간 우주 기업들의 

보다시피, 이제 AI어시스턴트는 전체 대화를 추적할 수 있다.

### 3. 비모수적 지식의 추가

개발에 있어서 학습에 사용된 모수적 지식 뿐만이 아닌, 사용자가 독자적으로 가지고 있는 비모수적 지식을 활용한다면 더 좋은 접근 방법이 될 수 있다. 이를 위해 문서를 vector DB에 임베딩하거나 검색기를 직접 사용하여 작업을 수행할 수 있다. 이 경우 `ConversationalRetrevalChain`을 사용하는 벡터 저장소 기반 검색기를 사용할 것이다. `ConversationalRetrevalChain`은 제공된 지식 베이스에 대한 검색기를 활용한다. 이 체인은 채팅 기록을 처리할 수 있으며, 이 채팅 기록은 이전에 설명한 원하는 유형의 메모리 (example : `ConversationBufferMemory`)를 사용해 매개변수로 전달된다. `ConversationalRetrevalChain`은 이러한 방식으로 현재의 대화 맥락을 고려하면서 지식 베이스에서 관련 정보를 검색할 수 있다.

이 예제를 위해 아래의 필요한 모든 요소를 초기화한다.

- document loader : 문서가 PDF형식이므로 `PyPDFLoader`를 사용한다
- text splitter : 지정된 문자들을 기준으로 텍스트를 재귀적으로 분할하는 `RecursiveCharacterTextSplitter`를 사용한다
- vector database : `FIASS` vector DB를 사용한다
- memory : `ConversationBufferMemory`를 사용한다
- LLM : 대화에는 `gpt-4o-mini` 모델을 사용한다
- embedding : `text-embedding-ada-002`를 사용한다

In [23]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)

raw_documents = PyPDFLoader('../data/portfolio_management_giude.pdf').load()
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

Ignoring wrong pointing object 53 0 (offset 0)


In [24]:
memory = ConversationBufferMemory(
    memory_key = 'chat_history',
    return_messages = True
)

In [25]:
llm = ChatOpenAI()

이제 체인과 상호 작용해 보자

In [26]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever = db.as_retriever(),
    memory = memory,
    verbose = True
)
qa_chain.invoke(
    {'question':'포트폴리오 운용 원칙에 대해서 설명해 줘'}
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Portfolio Management Guidelines  |  SBA  |  November 20204
   
1 Principles
Art.  1
The asset management agreement enables the bank to carry out all 
transactions it considers necessary in the context of regular asset 
management by banks. The bank pursues its mandate in good faith 
and in consideration of the client’s personal requirements that may 
reasonably be familiar to the bank. The bank acts on a discretionary 
basis in line with these guidelines, its asset allocation policy, the 
applicable investment strategy and any instructions set out by the 
client (including any investment restrictions). The asset management 
agreement does not authorise the bank to withdraw assets.
Implementing provis

{'question': '포트폴리오 운용 원칙에 대해서 설명해 줘',
 'chat_history': [HumanMessage(content='포트폴리오 운용 원칙에 대해서 설명해 줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='포트폴리오 운용 원칙은 은행이 자산 관리 업무를 성실하게 이행하도록 하는 지침입니다. 은행은 고객의 개인적 요구사항을 고려하며, 이에 따른 자산 배분 정책, 투자 전략, 고객의 지시사항 등을 준수하여 자산 운용을 진행합니다. 또한, 적절한 통제 및 모니터링 조치를 시행하여 관리 서비스를 고객에게 제공합니다. 이러한 원칙은 스위스은행협회의 지침으로, 2022년 1월 1일부터 적용됩니다.', additional_kwargs={}, response_metadata={})],
 'answer': '포트폴리오 운용 원칙은 은행이 자산 관리 업무를 성실하게 이행하도록 하는 지침입니다. 은행은 고객의 개인적 요구사항을 고려하며, 이에 따른 자산 배분 정책, 투자 전략, 고객의 지시사항 등을 준수하여 자산 운용을 진행합니다. 또한, 적절한 통제 및 모니터링 조치를 시행하여 관리 서비스를 고객에게 제공합니다. 이러한 원칙은 스위스은행협회의 지침으로, 2022년 1월 1일부터 적용됩니다.'}

`ConversationalRetrievalChain`은 `CONDENSE_QUESTION_PROMPT`라는 프롬프트 템플릿을 사용한다. 이 템플릿은 사용자의 마지막 질문을 채팅 기록과 병합해 검색기에 단일 쿼리로 전달한다. 사용자 정의 프롬프트를 전달하고 싶다면 `ConversationalRetrievalChain.from_llm` method의 `condense_question_prompt` parameter를 사용하면 된다.

봇이 문서를 기반으로 답변을 제공할 수 있었지만 여전히 한계가 존재한다. 실제로 이러한 구성에서는 사용자가 제공한 문서만 살펴보게 되는데, 봇이 모수적 지식도 활용하게 하려면 어떻게 해야 할까? 예를 들어서, 봇이 제공된 문서와 통합할 수 있는지 또는 단순히 '자유롭게'답변할 수 있는지를 이해할 수 있기를 원할 수 있다. 이를 위해서는 앱을 에이전트화 해야 하는데, 이는 LLM의 추론 기능을 활용하여 정해진 순서 없이 사용 가능한 도구를 조율하고 호출하는 대신 사용자의 질의에 따라 가장 적합한 접근 방식을 따르는 것을 의미한다. 이를 위해 두 가지 주요 구성 요소를 사용한다.

- `create_retriever_tool` : 이 메서드는 에이전트의 검색기 역할을 하는 사용자 지정 도구를 만든다. 검색할 데이터베이스, 이름, 간단한 설명이 있어야 모델이 그것들을 언제 사용할지 알 수 있다.
- `create_conversational_retrieval_agent` : 이 메서드는 검색기 및 채팅 모델과 함께 작동하도록 구성된 대화형 에이전트를 초기화한다. 여기에는 LLM, 도구 목록 (이 경우 검색기), 이전 채팅 기록을 추적하기 위한 메모리 키가 필요하다

아래의 코드는 에이전트를 초기화하는 방법을 설명한다

In [29]:
from langchain.prompts.prompt import PromptTemplate

custom_template = """다음 대화와 후속 질문을 바탕으로, 후속 질문을 독립적인 질문으로 다시 작성하세요.
제공된 문서에서 답을 찾을 수 없는 경우, 문서를 무시하고 어쨌든 답변하세요.
대화 기록:
{chat_history}
후속 입력: {question}
독립적인 질문:"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=db.as_retriever(), 
    condense_question_prompt=CUSTOM_QUESTION_PROMPT, 
    memory=memory, 
    verbose=True
)

qa_chain.invoke({'question': '위험선호형 포트폴리오를 구성하기 위해서는 어떻게 해야 하나요?'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
and other commodities, the bank has to ensure that no physical delivery 
is made to the client. 
15. Investments in non-traditional investments that do not comply with 
para. 13, their derivatives and combinations thereof are permissible for 
the purposes of portfolio diversification provided they are structured 
according to the Fund-of-Funds Principle (the fund is invested in several 
legally segregated collective investment schemes) or Multi-Manager-  
Principle (i. e., that the collective investment scheme’s portfolio is 
managed by more than one asset manager, each single asset manager 
being responsible solely for a specific part of the portfolio) or otherwise 
guarantee an equivalent diversifi

{'question': '위험선호형 포트폴리오를 구성하기 위해서는 어떻게 해야 하나요?',
 'chat_history': [HumanMessage(content='위험선호형 포트폴리오를 구성하기 위해서는 어떻게 해야 하나요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송하지만, 위험선호형 포트폴리오를 구성하는 방법에 대한 정보는 제공할 수 없습니다.', additional_kwargs={}, response_metadata={})],
 'answer': '죄송하지만, 위험선호형 포트폴리오를 구성하는 방법에 대한 정보는 제공할 수 없습니다.'}

In [34]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0
)

tool = create_retriever_tool(
    db.as_retriever(),
    'portfolio_management',
    '포트폴리오 운용에 관한 문서를 검색해서 반환합니다.'
)

tools = [tool]

memory = ConversationBufferMemory(
    memory_key = 'chat_history',
    return_messages = True,
    llm = llm
)

agent_executor = create_conversational_retrieval_agent(
    llm, tools,
    memory_key = 'chat_history',
    verbose = True
)

이제 두 가지 다른 질문으로 에이전트의 사고 과정을 살펴보자

In [35]:
agent_executor.invoke(
    {'input': '포트폴리오 운용 원칙에 대해서 설명해 줘'}
)



> Entering new AgentExecutor chain...

Invoking: `portfolio_management` with `{'query': '포트폴리오 운용 원칙'}`


Portfolio Management Guidelines  |  SBA  |  November 20204
   
1 Principles
Art.  1
The asset management agreement enables the bank to carry out all 
transactions it considers necessary in the context of regular asset 
management by banks. The bank pursues its mandate in good faith 
and in consideration of the client’s personal requirements that may 
reasonably be familiar to the bank. The bank acts on a discretionary 
basis in line with these guidelines, its asset allocation policy, the 
applicable investment strategy and any instructions set out by the 
client (including any investment restrictions). The asset management 
agreement does not authorise the bank to withdraw assets.
Implementing provisions
5. The bank establishes rules and guidelines regarding its overall invest -
ment approach, the investment process, the definition of eligible 
investment strategies, the investme

{'input': '포트폴리오 운용 원칙에 대해서 설명해 줘',
 'chat_history': [HumanMessage(content='포트폴리오 운용 원칙에 대해서 설명해 줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"포트폴리오 운용 원칙"}', 'name': 'portfolio_management'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 102, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'function_call', 'logprobs': None}, id='run-fe0836c0-e61e-45dd-9969-25c7b91773ca-0', usage_metadata={'input_tokens': 102, 'output_tokens': 24, 'total_tokens': 126, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  FunctionMessage(conte

이제 문서와 관련 없는 질문을 해 보자

In [36]:
output = agent_executor.invoke(
    {'input': '위험선호형 포트폴리오를 구성하고 관리하기 위해서 어떻게 해야 할까요?'}
)



> Entering new AgentExecutor chain...
위험선호형 포트폴리오를 구성하고 관리하기 위해서는 다음과 같은 단계와 전략을 고려해야 합니다:

1. **투자 목표 설정**:
   - 투자자의 목표와 시간 범위를 명확히 정의합니다. 예를 들어, 장기적인 자본 성장, 소득 창출, 또는 특정 재무 목표 달성을 위한 투자 등입니다.

2. **위험 수용 능력 평가**:
   - 투자자가 감수할 수 있는 위험 수준을 평가합니다. 이는 투자자의 재무 상황, 투자 경험, 심리적 요인 등을 고려하여 결정됩니다.

3. **자산 배분 전략 수립**:
   - 위험선호형 투자자는 일반적으로 주식, 고위험 자산, 대체 투자(예: 부동산, 원자재 등)에 더 많은 비중을 두는 경향이 있습니다. 자산 배분은 투자자의 목표와 위험 수용 능력에 따라 조정되어야 합니다.

4. **다양화**:
   - 포트폴리오 내에서 다양한 자산 클래스와 산업에 투자하여 위험을 분산시킵니다. 이는 특정 자산이나 산업의 부진이 전체 포트폴리오에 미치는 영향을 줄이는 데 도움이 됩니다.

5. **정기적인 모니터링 및 조정**:
   - 시장 상황, 경제 지표, 투자 성과 등을 정기적으로 모니터링하고 필요에 따라 포트폴리오를 조정합니다. 이는 자산 배분 비율을 재조정하거나, 새로운 투자 기회를 탐색하는 것을 포함합니다.

6. **리스크 관리**:
   - 손실을 제한하기 위한 전략(예: 손절매, 옵션 사용 등)을 수립합니다. 또한, 포트폴리오의 변동성을 관리하기 위해 헤지 전략을 고려할 수 있습니다.

7. **정보 수집 및 분석**:
   - 시장 동향, 경제 지표, 기업 실적 등을 지속적으로 분석하여 informed decision을 내립니다. 이를 통해 투자 전략을 조정하고 최적의 투자 결정을 내릴 수 있습니다.

8. **전문가의 조언 활용**:
   - 필요시 금융 전문가나 자산 관리 전문가의 조언을 받아 포트폴리오를 구성하고 관리하는 데 도움을 받을 수 있습니다.

위험

이 앱에 마지막으로 추가하고자 하는 기능은 웹 탐색 기능이다. 투자에 있어서 투자자들은 최신의 정보를 잡아내기 원할 것이기 때문이다. 랭체인의 도구를 사용해 이를 구현해 보자

### 4. 외부 도구 추가하기

이번에는 봇이 인터넷을 탐색할 수 있도록 `SerpApi` 도구를 추가한다. 앱이 웹에만 집중하게 하고 싶지는 않으므로, 앞에서 만든 도구와 이번에 새로 만들 `SerpApi` 도구를 함께 사용하도록 한다. 이렇게 하면 에이전트가 질문에 답하는 데 가장 유용한 도구를 선택하거나, 필요하지 않을 떄는 도구를 사용하지 않을 수도 있게 된다. 우선, 도구와 에이전트를 초기화해 보자.

In [37]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain_openai import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

import os
from dotenv import load_dotenv

load_dotenv()
os.environ['SERPAPI_API_KEY'] = serp_api_key

search = SerpAPIWrapper()

llm = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0
)

tools = [
    Tool.from_function(
        func = search.run,
        name = "Search",
        description = "현재 일어나고 있는 일에 관한 질문에 답할 때 유용합니다."
    ),
    create_retriever_tool(
        db.as_retriever(), 
        "portfolio_management",
        "포트폴리오 운용에 관한 문서를 검색해서 반환합니다."
    )
]

agent_executor = create_conversational_retrieval_agent(
    llm, 
    tools, 
    memory_key='chat_history', 
    verbose=True
)

In [38]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')

In [40]:
agent_executor.invoke({"input": "2025년 성장이 기대되는 미국 증권시장의 섹터나 테마는 무엇인가요?"})



> Entering new AgentExecutor chain...

Invoking: `Search` with `2025년 미국 증권시장 성장 기대 섹터 테마`


['AI는 단연 2025년에도 주목해야 할 테마입니다. 생성형 AI 기술이 텍스트, 이미지, 영상 생성 능력을 보여주며 많은 관심을 받고 있습니다. AI 시장은 ...', '주요 경제 전망 ... 미 연준(Fed)의 긴축 통화 정책 여파가 지연 효과로 나타나면서, 미국 경제는 2025년 약 2.0%의 추세적 성장률을 기록할 것으로 전망됩니다. 근원 ...', 'S&P 500 지수는 2023년 24.2% 오른 데 이어 지난해에도 AI(인공지능) 붐에 힘입어 23.3% 상승했다. 2년간 무려 53% 올랐다. 인터넷 붐이 일었던 1997~1998 ...', '차기 미 대통령 선거 결과가 추가 상승 동력이 되어 다우존스를 비롯한 미국의 주요 주식시장 지수들이 사상 최고가를 기록하며 2025년을 준비하고 있는 ...', '담당: 임승미 연구위원(02-3771-7786) / 강윤형 연구원(02-3771-8024) / 박준우 연구원(02-3771-8116). 하나증권 리서치센터 글로벌투자분석실.', '삼성자산운용 관계자도 "미국 빅테크들의 AI 투자가 지속될 것"이라며 "2025년에도 미국이 주도하는 AI 기술 혁신은 이어질 것으로 전망한다"고 전망했다.', '알렉스 테더(Alex Tedder)와 팀 윌슨(Tom Wilson)이 주식 수익률 기회와 관련하여 시장 환경의 변화에 따라 주목할 섹터와 국가, 테마를 탐색합니다.', '미국 ETF 시장의 성장 배경은 1) 뮤추얼펀드 대비 유리. 한 세금 효율성, 2) 뮤추얼펀드 대비 낮은 운용 보수, 3). ETF 다양화로 투자자들의 선택지 확대 ...']2025년 미국 증권시장에서 성장이 기대되는 섹터나 테마는 다음과 같습니다:

1. **인공지능(AI)**: 생성형 AI 기술이 텍스트, 이미지, 영상 생성 능력을 보여주며 많은 관심을 받고 있습니다. AI 시장

{'input': '2025년 성장이 기대되는 미국 증권시장의 섹터나 테마는 무엇인가요?',
 'chat_history': [HumanMessage(content='2025년 성장이 기대되는 섹터나 테마는 무엇인가요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"2025년 성장 기대 섹터 테마"}', 'name': 'Search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 142, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-fdbec66d-75c5-4826-9269-ecd79ff49aef-0', usage_metadata={'input_tokens': 142, 'output_tokens': 25, 'total_tokens': 167, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  Funct

In [41]:
agent_executor.invoke({"input": "트럼프가 2024년 미국 대선에서 승리하게 되었는데, 이로 인해 테슬라와 암호화폐 관련 주가가 상승하는 등의 격변이 보이고 있습니다. 양자컴퓨팅 테마가 급상승하며 차후 2025년 미국 증시의 테마가 어떻게 될지 모두가 주목을 하고 있는데, 안정적인 포트폴리오를 운영하기 위해서 2025년에 종목 구성을 어떻게 하는게 좋을까요? 목표 수익률은 연간 15%입니다."})



> Entering new AgentExecutor chain...

Invoking: `portfolio_management` with `{'query': '2025년 안정적인 포트폴리오 구성 종목 추천 목표 수익률 15%'}`


and other commodities, the bank has to ensure that no physical delivery 
is made to the client. 
15. Investments in non-traditional investments that do not comply with 
para. 13, their derivatives and combinations thereof are permissible for 
the purposes of portfolio diversification provided they are structured 
according to the Fund-of-Funds Principle (the fund is invested in several 
legally segregated collective investment schemes) or Multi-Manager-  
Principle (i. e., that the collective investment scheme’s portfolio is 
managed by more than one asset manager, each single asset manager 
being responsible solely for a specific part of the portfolio) or otherwise 
guarantee an equivalent diversification.

Portfolio Management Guidelines  |  SBA  |  November 20208
  
16. The asset management mandate does not entitle the bank to grant 
corporate loans to

{'input': '트럼프가 2024년 미국 대선에서 승리하게 되었는데, 이로 인해 테슬라와 암호화폐 관련 주가가 상승하는 등의 격변이 보이고 있습니다. 양자컴퓨팅 테마가 급상승하며 차후 2025년 미국 증시의 테마가 어떻게 될지 모두가 주목을 하고 있는데, 안정적인 포트폴리오를 운영하기 위해서 2025년에 종목 구성을 어떻게 하는게 좋을까요? 목표 수익률은 연간 15%입니다.',
 'chat_history': [HumanMessage(content='2025년 성장이 기대되는 미국 증권시장의 섹터나 테마는 무엇인가요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"2025년 미국 증권시장 성장 기대 섹터 테마"}', 'name': 'Search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 455, 'total_tokens': 484, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-1db58d80-fb77-40cf-85bd-640f50a9a61a-0', usage_metadata={

이제 앱은 최신 정보를 제공할 뿐만 아니라, 선별된 문서에서 특정 지식을 검색할 수 있게 되었다. 다음 단계는 frontend를 구축하는 것이다.